# Introduction to Decision Trees
Author: Hovanes Gasparian

*Some materials adapted from Matt Brems and David Yerrington from General Assembly and Victor Zhou from Facebook*

### Learning Objectives

- Explain how a decision tree functions
- Learn how to build a decision tree

## What will we wear today?

|$Y = $ Clothing|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Coat  |      Rainy     |   Weekday  |
|   T-Shirt   |      Sunny     |   Weekday  |
|   Coat  |      Rainy     |   Weekend  |
|  Hoodie  |      Sunny     |   Weekend  |
|   Coat  |      Rainy     |   Weekday  |
|  Hoodie  |      Sunny     |   Weekend  |

---

<details><summary>It's a rainy day. Based on our past, what do you think we'll wear?</summary>

- A coat.
- In 100% of past cases where the weather is rainy, we've worn a coat!

|$Y = $ Clothing|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   Coat  |      Rainy     |   Weekday  |
|   Coat  |      Rainy     |   Weekend  |
|   Coat  |      Rainy     |   Weekday  |

</details>

---

<details><summary>It's a sunny day. Based on our past, what do you think we'll wear?</summary>

- Either a T-Shirt or a Hoodie... Based on our past, we've worn T-Shirts on 1/3 of sunny days and we wore Hoodies on 2/3 of sunny days.
- If we **had** to make an educated guess, we'd probably go with Hoodie, because that's more likely given just this information.

|$Y = $ Clothing|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|   T-Shirt   |      Sunny     |   Weekday  |
|  Hoodie  |      Sunny     |   Weekend  |
|  Hoodie  |      Sunny     |   Weekend  |

</details>

---

<details><summary>It's a sunny day that also happens to be a weekend. Based on our past, what do you think we'll wear?</summary>

- A Hoodie.
- In 100% of past cases where the weather is sunny and where it's a weekend, we've worn a hoodie!

|$Y = $ Clothing|$X_1 = $ Weather|$X_2 = $ Day|
|:---------:|:--------------:|:----------:|
|  Hoodie  |      Sunny     |   Weekend  |
|  Hoodie  |      Sunny     |   Weekend  |

</details>

# Decision Trees: Overview

A decision tree:
- takes a dataset consisting of $X$ and $Y$ data, 
- finds rules based on our $X$ data that splits the data into smaller subsets so that at the end, the values $Y$ in each "leaf node" are as "pure" as possible.

Decision trees are often represented by a graph.

<img src="./DecisionTree.png" alt="what_to_wear" width="850"/>

- (This image was created using [Draw.io](https://www.draw.io/).)

---



Decision trees are aptly named, because they look like upside down trees. 
- What you see on top is known as the "root node," through which all observations are passed.
- At each internal split, the dataset is partitioned.
- At each of the "leaf nodes" (i.e. the clothing boxes), we have a partition of observations that are as pure as possible.
    - In this clothing example, each leaf node is perfectly pure. Once we get to a leaf node, every observation in that leaf node has the exact same value of $Y$!
    - There are ways to quantify this idea of "purity," and how the underlying algorithm decides where to make splits, which is demonstrated in the handout I gave you in class.

For your reference, decision trees are technically referred to as "**Classification and Regression Trees**," abbreviated as "**CART**."
- [DecisionTreeClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [DecisionTreeRegressor Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [1]:
import numpy as np

## Purity in Decision Trees

When quantifying how "pure" a node is, we want to see what the distribution of $Y$ is in each node, then summarize this distribution with a number.

- For continuous $Y$ (i.e. using a decision tree to predict income), the default option is mean squared error.
    - When the decision tree is figuring out which split to make at a given node, it picks the split that minimizes the MSE at that step.
    
    
- For discrete $Y$, the default option is the Gini impurity. (This is not quite the same thing as the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient).)

$$
\begin{eqnarray*}
\text{Gini} &=& 1 - \sum_{i=1}^{classes} P(\text{class i})^2 \\
\text{Gini (2 classes)} &=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 \\
\text{Gini (3 classes)} &=& 1 - P(\text{class 1})^2 - P(\text{class 2})^2 - P(\text{class 3})^2 \\
\end{eqnarray*}
$$

- We use Gini to measure how pure a split is.
- Gini impurity ranges from 0 to 0.5  when you have 2 classes (binary case), where:
    - 0 means "most pure."
    - 0.5 means "least pure."
    
- Gini impurity (with $k$ classes) ranges from 0 to $1-\frac{1}{k}$.

In [2]:
# Create Y variable from the previous clothing example
Y = ['Coat', 'T-Shirt', 'Coat', 'Hoodie', 'Coat', 'Hoodie']

In [3]:
# Define Gini function.
def gini(y):
    # create an empty list to store squared probabilities
    gini_sum = []

    # iterate through my classes, without double-counting
    for i in set(y):
        # calculate probability
        prob = y.count(i) / len(y)
        
        # append squared proability
        gini_sum.append(prob**2)
    
    # return Gini impurity
    return 1 - sum(gini_sum)

In [4]:
# Check gini impurity of current Y variable
gini(Y)

0.6111111111111112

<details><summary>Under what circumstances should our Gini be 0?</summary>
- When all of our observations are identical.
</details>

In [5]:
# Under what circumstances should our Gini be 0?
gini(['Coat', 'Coat', 'Coat'])

0.0

<details><summary>Under what circumstances should our Gini be 0.5?</summary>
- When we have exactly two outcomes that are equally represented.
</details>

In [6]:
# Under what circumstances should our Gini be 0.5?
gini(['T-Shirt', 'Hoodie'])

0.5

### How do decision trees use Gini to decide where to make a split?

- At any node, it considers the subset of observations that exist at that node.
- It iterates through each variable that could potentially split the data.
- It then calculates the Gini impurity that results from every split.
- Finally, it selects the variable that causes the greatest decrease in Gini impurity from the parent node to the child node.

Optimization occurs at each node individually, making this a **greedy** algorithm. This is one of the disadvantages that will be discussed more at the end.

## Building a Decision Tree

In [7]:
# Import pandas
import pandas as pd

# Read in Breast Cancer data.
bcdf = pd.read_csv('./breast-cancer.csv', header=None)

# create column names from header file
field_names = open('./field_names.txt').read().splitlines()

# set column names as field_names
bcdf.columns = field_names

In [8]:
# Create outcome variable for malignant tumors
bcdf['malignant'] = pd.get_dummies(bcdf.diagnosis)['M']
y = bcdf.malignant

# Create independent variables
X = bcdf.drop(columns=['diagnosis', 'ID', 'malignant'])

# Conduct train/test split.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state = 42)

In [9]:
# Import decision tree model
from sklearn.tree import DecisionTreeClassifier

In [10]:
# Instantiate the model
dt = DecisionTreeClassifier(random_state=42)

In [11]:
# Fit model.
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [12]:
# Evaluate model.
print(f'Training Score: {dt.score(X_train, y_train)}')

Training Score: 1.0


In [13]:
print(f'Testing Score: {dt.score(X_test, y_test)}')

Testing Score: 0.9415204678362573


<details><summary>What conclusion do we usually draw when we see this kind of result?</summary>

- The model is overfit to the data.
</details>

<details><summary>What might we do to alleviate this situation?</summary>
    
- Try to get more data.
- Remove some features.
- Tune your hyperparameters
</details>

---

# Conclusion

## Advantages

### 1. No need to standardize
 - Decision trees don't rely on scale to be effective.

### 2. Decision trees don't care about how your data is distributed.

 - Decision trees are nonparametric, meaning we don't make assumptions about how our data or errors are distributed. Thus, they are effective even if your data is heavily skewed or not normally distributed. 

### 3. Easy to interpret.

 - The output of a decision tree is easy to interpet and and there is even a built-in method for showing feature importance

### 4. Speed.

 - Decision tree models are very fast!

## Disadvantages


### 1. Decision trees are prone to overfitting.
 - But we already discussed some ways to mitigate that, and there are others.

### 2. Decision trees are locally optimal.
 - Because we're making the best decision at each local node (i.e. greedy search), we might end up with a less than optimal outcome.

### 3. Decision trees don't work well with unbalanced data.
 - As with most classification methods, decision trees also bias results toward the majority class. There are workarounds for this too that we will discuss in future lessons!